In [1]:
install.packages("disk.frame", repo="https://cran.rstudio.com/")
install.packages("xgboost")
install.packages("Ckmeans.1d.dp")
library(tidyverse)
library(xgboost)  # the main algorithm
#library(archdata) # for the sample dataset
library(caret)    # for the confusionmatrix() function (also needs e1071 package)
library(dplyr)    # for some data preperation
library(Ckmeans.1d.dp) # for xgb.ggplot.importance
library(caret)
library(nnet)
library(reticulate)
sagemaker <- import('sagemaker')
session <- sagemaker$Session()
bucket <- session$default_bucket()

also installing the dependencies ‘pryr’, ‘fst’

Warning message in install.packages("disk.frame", repo = "https://cran.rstudio.com/"):
“installation of package ‘pryr’ had non-zero exit status”Warning message in install.packages("disk.frame", repo = "https://cran.rstudio.com/"):
“installation of package ‘fst’ had non-zero exit status”Warning message in install.packages("disk.frame", repo = "https://cran.rstudio.com/"):
“installation of package ‘disk.frame’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ────────────────────────

In [2]:
# Load the data
triagedata <- get(load(file = "processedData5.RData"))
#head(data)
#data <- as.dataframe(data)
#sapply(data, class)
str(triagedata)
summary(triagedata)
dim(triagedata)

tibble [556,298 × 107] (S3: tbl_df/tbl/data.frame)
 $ PRESENTING_PROBLEM                      : Factor w/ 220 levels "Abdominal distension",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ PATIENT_ID                              : Factor w/ 368472 levels "0000196c5c11094e3014b171ea3e1846d3dcd8ef",..: 307925 201284 169928 296938 66423 219078 293523 20566 317304 293359 ...
 $ AGE                                     : num [1:556298] 50 49 55 62 50 75 46 69 70 86 ...
 $ ADMISSION_WITHIN_48_HOURS               : num [1:556298] 0 0 0 0 0 0 0 0 0 0 ...
 $ TRIAGE_DT_TM                            : POSIXct[1:556298], format: "2019-08-04 02:50:00" "2018-01-05 00:52:00" ...
 $ TRIAGE_CATEGORY                         : Factor w/ 5 levels "1","2","3","4",..: 3 3 2 3 3 3 3 2 4 3 ...
 $ TRIAGE_PRESENTING_INFORMATION           : chr [1:556298] "abvdo distension for last 3 days. no vomting pt has peg tube insitu. has traumatic brain injury was due to go t"| __truncated__ "GP referal with low Hb. Saw GP as was feeling le

                         PRESENTING_PROBLEM
 Pain, abdominal                  : 58690  
 Pain, chest                      : 48055  
 Unwell                           : 26951  
 Respiratory - shortness of breath: 25140  
 Care - patient review            : 22087  
 (Other)                          :372531  
 NA's                             :  2844  
                                    PATIENT_ID          AGE        
 be763e67f4a28e30f71cdbadaea4813bd28cdda8:   234   Min.   :  0.00  
 a40ac0bb8fb629b54d4d87678a65f8243d169405:   184   1st Qu.: 24.00  
 40afc5643936dfe8f32373af04360fce28ecaa24:   157   Median : 39.00  
 34591ed9ac883d57997d3b19563dac8956909b0e:   138   Mean   : 41.94  
 d4ef8dd7aa81f5f363335eb227f892bc7cd3094a:   129   3rd Qu.: 61.00  
 961cc31088f2a6d5b51efd07d594a4089b62b7ad:   124   Max.   :107.00  
 (Other)                                 :555332                   
 ADMISSION_WITHIN_48_HOURS  TRIAGE_DT_TM                 TRIAGE_CATEGORY
 Min.   :0.00000           Min.

[1] 556298    107

In [3]:
library(dplyr)
#ED_data = select(data,-ID)
triagedata$ID <- NULL
ED_data = triagedata %>% 
  #select(-ends_with('_2')) %>%
  select(-ends_with('1_DT_TM')) %>%
  select(-ends_with('2_DT_TM')) %>%
  select(-c(AGE, PRESENTING_PROBLEM, PATIENT_ID,TRIAGE_DT_TM)) %>%
  select(-c(TRIAGE_PRESENTING_INFORMATION,TRIAGE_PRESENTING_ADDITIONAL_INFORMATION,PATIENT_PROBLEM_HISTORY)) %>%
  select(TRIAGE_CATEGORY, everything()) 
ED_data$TRIAGE_CATEGORY = as.numeric(ED_data$TRIAGE_CATEGORY)
str(ED_data)
summary(ED_data)

tibble [556,298 × 75] (S3: tbl_df/tbl/data.frame)
 $ TRIAGE_CATEGORY             : num [1:556298] 3 3 2 3 3 3 3 2 4 3 ...
 $ ADMISSION_WITHIN_48_HOURS   : num [1:556298] 0 0 0 0 0 0 0 0 0 0 ...
 $ AVPU_1                      : num [1:556298] 1 1.04 1.04 1 1 ...
 $ AVPU_2                      : num [1:556298] 1 0 0 1 1 1 1 1 1 1 ...
 $ GCS_1                       : num [1:556298] 14.9 14.9 14.9 15 14.9 ...
 $ GCS_2                       : num [1:556298] 0 0 0 15 0 0 0 14 15 15 ...
 $ RR_1                        : num [1:556298] 18 18.8 24 20 18 ...
 $ RR_2                        : num [1:556298] 18 0 0 18 18 18 15 34 20 19 ...
 $ O2SATS_1                    : num [1:556298] 100 98.1 91 93 99 ...
 $ O2SATS_2                    : num [1:556298] 100 0 0 93 99 97 97 96 99 100 ...
 $ FIO2_1                      : num [1:556298] 35.8 43 39.6 710.4 39.6 ...
 $ FIO2_2                      : num [1:556298] 0 0 0 0 0 0 0 22 0 0 ...
 $ PULSE_1                     : num [1:556298] 97 86.4 108 85 75

 TRIAGE_CATEGORY ADMISSION_WITHIN_48_HOURS     AVPU_1          AVPU_2      
 Min.   :1.000   Min.   :0.00000           Min.   :1.000   Min.   :0.0000  
 1st Qu.:2.000   1st Qu.:0.00000           1st Qu.:1.000   1st Qu.:0.0000  
 Median :3.000   Median :0.00000           Median :1.000   Median :0.0000  
 Mean   :3.156   Mean   :0.06311           Mean   :1.034   Mean   :0.4084  
 3rd Qu.:4.000   3rd Qu.:0.00000           3rd Qu.:1.034   3rd Qu.:1.0000  
 Max.   :5.000   Max.   :1.00000           Max.   :5.000   Max.   :5.0000  
     GCS_1           GCS_2             RR_1             RR_2       
 Min.   : 3.00   Min.   : 0.000   Min.   :  0.00   Min.   :  0.00  
 1st Qu.:14.87   1st Qu.: 0.000   1st Qu.: 18.00   1st Qu.:  0.00  
 Median :14.94   Median : 0.000   Median : 18.82   Median : 16.00  
 Mean   :14.86   Mean   : 2.765   Mean   : 20.62   Mean   : 10.73  
 3rd Qu.:15.00   3rd Qu.: 0.000   3rd Qu.: 20.00   3rd Qu.: 18.00  
 Max.   :15.00   Max.   :15.000   Max.   :249.00   Max.   :1

In [4]:
ED_data <- as.data.frame(ED_data)
str(ED_data)

'data.frame':	556298 obs. of  75 variables:
 $ TRIAGE_CATEGORY             : num  3 3 2 3 3 3 3 2 4 3 ...
 $ ADMISSION_WITHIN_48_HOURS   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ AVPU_1                      : num  1 1.04 1.04 1 1 ...
 $ AVPU_2                      : num  1 0 0 1 1 1 1 1 1 1 ...
 $ GCS_1                       : num  14.9 14.9 14.9 15 14.9 ...
 $ GCS_2                       : num  0 0 0 15 0 0 0 14 15 15 ...
 $ RR_1                        : num  18 18.8 24 20 18 ...
 $ RR_2                        : num  18 0 0 18 18 18 15 34 20 19 ...
 $ O2SATS_1                    : num  100 98.1 91 93 99 ...
 $ O2SATS_2                    : num  100 0 0 93 99 97 97 96 99 100 ...
 $ FIO2_1                      : num  35.8 43 39.6 710.4 39.6 ...
 $ FIO2_2                      : num  0 0 0 0 0 0 0 22 0 0 ...
 $ PULSE_1                     : num  97 86.4 108 85 75 ...
 $ PULSE_2                     : num  98 0 0 84 78 66 99 97 75 75 ...
 $ HEART_RATE_1                : num  81.8 81.7 80.8 81.1 80.

In [5]:
# Convert the Species factor to an integer class starting at 0
# This is picky, but it's a requirement for XGBoost
#TRIAGE_CATEGORY = ED_data$TRIAGE_CATEGORY
#label = as.integer(ED_data$TRIAGE_CATEGORY)-1
#ED_data$TRIAGE_CATEGORY = NULL

In [6]:
# Make split index
train_index <- sample(1:nrow(ED_data), nrow(ED_data)*0.8)
# Full data set
data_variables <- as.matrix(ED_data[,-1])
data_label <- ED_data[,"TRIAGE_CATEGORY"]
data_matrix <- xgb.DMatrix(data = as.matrix(ED_data), label = data_label)
# split train data and make xgb.DMatrix
train_data   <- data_variables[train_index,]
train_label  <- data_label[train_index]
train_matrix <- xgb.DMatrix(data = train_data, label = train_label)
# split test data and make xgb.DMatrix
test_data  <- data_variables[-train_index,]
test_label <- data_label[-train_index]
test_matrix <- xgb.DMatrix(data = test_data, label = test_label)

In [8]:
numberOfClasses <- length(unique(ED_data$TRIAGE_CATEGORY))
xgb_params <- list(objective = "multi:softprob",
                   eval_metric = "mlogloss",
                   num_class = numberOfClasses)
#cv.nround <- 50 # number of XGBoost rounds
#cv.nfold  <- 5

# Fit cv.nfold * cv.nround XGB models and save OOF predictions
cv_model <- xgb.cv(params = xgb_params,
                   data = train_matrix, 
                   nrounds = 1000,
                   nfold = 10,
                   verbose = 0,
                   prediction = TRUE)

ERROR: Error in xgb.iter.update(fd$bst, fd$dtrain, iteration - 1, obj): [05:30:31] amalgamation/../src/objective/multiclass_obj.cu:115: SoftmaxMultiClassObj: label must be in [0, num_class).
Stack trace:
  [bt] (0) /home/ec2-user/anaconda3/envs/R/lib/R/library/xgboost/libs/xgboost.so(+0x7d8bf) [0x7f9ad99c78bf]
  [bt] (1) /home/ec2-user/anaconda3/envs/R/lib/R/library/xgboost/libs/xgboost.so(+0xe92b6) [0x7f9ad9a332b6]
  [bt] (2) /home/ec2-user/anaconda3/envs/R/lib/R/library/xgboost/libs/xgboost.so(+0x12b6a6) [0x7f9ad9a756a6]
  [bt] (3) /home/ec2-user/anaconda3/envs/R/lib/R/library/xgboost/libs/xgboost.so(XGBoosterUpdateOneIter+0x3c) [0x7f9ad99eb90c]
  [bt] (4) /home/ec2-user/anaconda3/envs/R/lib/R/library/xgboost/libs/xgboost.so(XGBoosterUpdateOneIter_R+0x41) [0x7f9ad99c4f21]
  [bt] (5) /home/ec2-user/anaconda3/envs/R/lib/R/bin/exec/../../lib/libR.so(+0x13d548) [0x7f9aee570548]
  [bt] (6) /home/ec2-user/anaconda3/envs/R/lib/R/bin/exec/../../lib/libR.so(Rf_eval+0x190) [0x7f9aee57a690]
  [

In [ ]:
OOF_prediction <- data.frame(cv_model$pred) %>%
  mutate(max_prob = max.col(., ties.method = "last"),
         label = train_label + 1)
head(OOF_prediction)

In [ ]:
confusionMatrix(factor(OOF_prediction$max_prob),
                factor(OOF_prediction$label),
                mode = "everything")

In [ ]:
bst_model <- xgb.train(params = xgb_params,
                       data = train_matrix,
                       nrounds = nround)

# Predict hold-out test set
test_pred <- predict(bst_model, newdata = test_matrix)
test_prediction <- matrix(test_pred, nrow = numberOfClasses,
                          ncol=length(test_pred)/numberOfClasses) %>%
  t() %>%
  data.frame() %>%
  mutate(label = test_label + 1,
         max_prob = max.col(., "last"))
# confusion matrix of test set
confusionMatrix(factor(test_prediction$max_prob),
                factor(test_prediction$label),
                mode = "everything")

In [ ]:
# get the feature real names
names <-  colnames(ED_data[,-1])
# compute feature importance matrix
importance_matrix = xgb.importance(feature_names = names, model = bst_model)
head(importance_matrix)

In [ ]:
gp = xgb.ggplot.importance(importance_matrix)
print(gp)